# A full training

How to achieve the same results without using the Trainer class?

In [1]:
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding

raw_datasets = load_dataset("glue", "mrpc")
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

def tokenize_function(example):
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True)

tokenized_dataset = raw_datasets.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

c:\Users\IdeaPad\anaconda3\envs\pytorch\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Prepare for training

In [2]:
tokenized_dataset = tokenized_dataset.remove_columns(["sentence1", "sentence2", "idx"])
tokenized_dataset = tokenized_dataset.rename_column("label", "labels")
tokenized_dataset.set_format("torch")
tokenized_dataset["train"].column_names

['labels', 'input_ids', 'token_type_ids', 'attention_mask']

In [3]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(tokenized_dataset["train"], shuffle=True, batch_size=8, collate_fn=data_collator)
eval_dataloader = DataLoader(tokenized_dataset["validation"], shuffle=True, batch_size=8, collate_fn=data_collator)

In [4]:
for batch in train_dataloader:
    break

{k: v.shape for k,v in batch.items()}

{'labels': torch.Size([8]),
 'input_ids': torch.Size([8, 53]),
 'token_type_ids': torch.Size([8, 53]),
 'attention_mask': torch.Size([8, 53])}

In [5]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
outputs = model(**batch)
print(outputs.loss, outputs.logits.shape)

tensor(0.7275, grad_fn=<NllLossBackward0>) torch.Size([8, 2])


In [7]:
from transformers import AdamW

optimizer = AdamW(model.parameters(), lr=5e-5)

c:\Users\IdeaPad\anaconda3\envs\pytorch\Lib\site-packages\transformers\optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [8]:
from transformers import get_scheduler

num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)

print(num_training_steps)

1377


### The training loop

In [9]:
import torch

device = torch.device("xpu") if torch.xpu.is_available() else torch.device("cpu")
model.to(device)
device

device(type='xpu')

In [ ]:
from tqdm.auto import tqdm

progress_bar = tqdm(range(num_training_steps))

model.train()
for epoch in range(num_epochs):
    for batch in train_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

### The Evaluation loop

In [10]:
import evaluate

metric = evaluate.load("glue", "mrpc")
model.eval()
for batch in eval_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    metric.add_batch(predictions=predictions, references=batch["labels"])

metric.compute()

c:\Users\IdeaPad\anaconda3\envs\pytorch\Lib\site-packages\torchvision\io\image.py:14: UserWarning: Failed to load image Python extension: 'Could not find module 'C:\Users\IdeaPad\anaconda3\envs\pytorch\Lib\site-packages\torchvision\image.pyd' (or one of its dependencies). Try using the full path with constructor syntax.'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


{'accuracy': 0.3161764705882353, 'f1': 0.0}

## SST Dataset

In [12]:
sst_dataset = load_dataset("glue", "sst2")

In [47]:
def tokenize_dataset(example):
    return tokenizer(example["sentence"])

In [48]:
tokenized_dataset = sst_dataset.map(tokenize_dataset, batched=True)

In [49]:
tokenized_dataset = tokenized_dataset.remove_columns(["idx","sentence"])
tokenized_dataset = tokenized_dataset.rename_column("label", "labels")
tokenized_dataset.set_format("torch")
tokenized_dataset


DatasetDict({
    train: Dataset({
        features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 67349
    })
    validation: Dataset({
        features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 872
    })
    test: Dataset({
        features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1821
    })
})

In [50]:
collator_fn = DataCollatorWithPadding(tokenizer=tokenizer)

In [51]:
train_loader = DataLoader(tokenized_dataset["train"], shuffle=True, batch_size=8, collate_fn=collator_fn)
eval_dataloader = DataLoader(tokenized_dataset["validation"], shuffle=True, batch_size=8, collate_fn=collator_fn)

In [52]:
batch = next(iter(train_loader))
model.to("cpu")

output = model(**batch)
output

SequenceClassifierOutput(loss=tensor(0.6622, grad_fn=<NllLossBackward0>), logits=tensor([[ 0.1145,  0.0777],
        [-0.0384,  0.0935],
        [-0.0543,  0.1016],
        [ 0.0842,  0.0377],
        [-0.0822,  0.2762],
        [ 0.0603, -0.2363],
        [-0.1340,  0.1481],
        [ 0.0133, -0.1489]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [34]:
model.to(device)
model.train()

optimizer = AdamW(model.parameters(), lr=5e-5)

num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)

print(num_training_steps)

from tqdm.auto import tqdm

progress_bar = tqdm(range(num_training_steps))

c:\Users\IdeaPad\anaconda3\envs\pytorch\Lib\site-packages\transformers\optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


1377


  0%|          | 0/1377 [00:00<?, ?it/s]

In [ ]:

for epoch in range(num_epochs):
    for batch in train_loader:
        batch = {k:v.to(device) for k,v in batch.items()}
        output = model(**batch)

        output.loss.backward()
        optimizer.step()

        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)


In [54]:
model.to(device)
model.eval()

for batch in eval_dataloader:
    batch = {k:v.to(device) for k,v in batch.items()}
    with torch.no_grad():
        output = model(**batch)
    logits = output.logits
    preds = torch.argmax(logits, dim=-1)
    metric.add_batch(predictions=preds, references=batch["labels"])
metric.compute()

{'accuracy': 0.7041284403669725, 'f1': 0.7552182163187856}

### Supercharge your PyTorch training loop with Accelerate

Using the 🤗 Accelerate library, with just a few adjustments we can enable distributed training on multiple GPUs or TPUs

In [ ]:
from accelerate import Accelerator
from transformers import AdamW, AutoModelForSequenceClassification, get_scheduler

accelerator = Accelerator()

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)
optimizer = AdamW(model.parameters(), lr=3e-5)

train_dl, eval_dl, model, optimizer = accelerator.prepare(
    train_dataloader, eval_dataloader, model, optimizer
)

num_epochs = 3
num_training_steps = num_epochs * len(train_dl)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)

progress_bar = tqdm(range(num_training_steps))

model.train()
for epoch in range(num_epochs):
    for batch in train_dl:
        outputs = model(**batch)
        loss = outputs.loss
        accelerator.backward(loss)

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)